In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/ml-latest

/content/gdrive/My Drive/ml-latest


In [3]:
%ls

genome-scores.csv  movies.csv   reco_NeuralNmodel.h5  test.csv
genome-tags.csv    ratings.csv  reco_NNmodel.h5       test_Neural.csv
links.csv          README.txt   tags.csv


In [0]:
import pandas as pd

In [5]:
orig_df = pd.read_csv("ratings.csv")
orig_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
orig_df['movieIdCat'] = orig_df.movieId.astype('category').cat.codes.values
orig_df['userIdCat'] = orig_df.userId.astype('category').cat.codes.values
orig_df.head()

,userId,movieId,rating,timestamp,movieIdCat,userIdCat
0,1,307,3.5,1256677221,304,0
1,1,481,3.5,1256677456,477,0
2,1,1091,1.5,1256677471,1069,0
3,1,1257,4.5,1256677460,1229,0
4,1,1449,4.5,1256677264,1414,0


In [0]:
idx = orig_df.groupby('userId').apply(lambda x: x.sample(frac=0.8, random_state = 0)).index.get_level_values(1)

train = orig_df.iloc[idx, :].reset_index(drop = True)
test = orig_df.drop(idx).reset_index(drop = True)

In [8]:
train.head(100)

,userId,movieId,rating,timestamp,movieIdCat,userIdCat
0,1,481,3.5,1256677456,477,0
1,1,1591,1.5,1256677475,1539,0
2,1,2478,4.0,1256677239,2394,0
3,1,2840,3.0,1256677500,2755,0
4,1,3698,3.5,1256677243,3608,0
...,...,...,...,...,...,...
95,4,8622,4.0,1113796513,7939,3
96,4,6104,4.5,1113796011,6006,3
97,4,2672,4.5,1114050326,2587,3
98,4,7573,4.0,1114050840,7376,3


In [9]:
test.head(100)

,userId,movieId,rating,timestamp,movieIdCat,userIdCat
0,1,307,3.5,1256677221,304,0
1,1,1590,2.5,1256677236,1538,0
2,1,3424,4.5,1256677444,3336,0
3,2,170,3.5,1192913581,168,1
4,2,1296,4.5,1192913608,1267,1
...,...,...,...,...,...,...
95,4,3190,3.5,1113766176,3104,3
96,4,3264,3.5,1145937802,3178,3
97,4,3354,3.5,1113796563,3268,3
98,4,3388,3.0,1113766172,3301,3


In [10]:
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
n_users, n_movies = len(orig_df.userIdCat.unique()), len(orig_df.movieIdCat.unique())
n_latent_factors = 3

Using TensorFlow backend.


In [0]:
n_latent_factors_user = 5
n_latent_factors_movie = 8

movie_input = keras.layers.Input(shape=[1],name='Movie')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([movie_vec, user_vec], axis=-1)
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 8)         431120      Movie[0][0]                      
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         1416145     User[0][0]                       
____________________________________________________________________________________________

In [13]:
history = model.fit([train.userIdCat, train.movieIdCat], train.rating, epochs=10, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
22207659/22207659 [==============================] - 3365s 152us/step - loss: 0.6870
Epoch 2/10
22207659/22207659 [==============================] - 3319s 149us/step - loss: 0.6717
Epoch 3/10
22207659/22207659 [==============================] - 3347s 151us/step - loss: 0.6681
Epoch 4/10
22207659/22207659 [==============================] - 3347s 151us/step - loss: 0.6658
Epoch 5/10
22207659/22207659 [==============================] - 3390s 153us/step - loss: 0.6647
Epoch 6/10
22207659/22207659 [==============================] - 3439s 155us/step - loss: 0.6637
Epoch 7/10
22207659/22207659 [==============================] - 3440s 155us/step - loss: 0.6630
Epoch 8/10
22207659/22207659 [==============================] - 3397s 153us/step - loss: 0.6623
Epoch 9/10
22207659/22207659 [==============================] - 3316s 149us/step - loss: 0.6619
Epoch 10/10
22207659/22207659 [==============================] - 3338s 150us/step - loss: 0.6614


In [0]:
model.save('reco_NNmodel.h5')

In [0]:
model.save('reco_NeuralNmodel.h5')

In [0]:
test.head()
test.to_csv('test_Neural.csv') 

In [5]:
from keras.models import load_model
model = load_model('reco_NeuralNmodel.h5')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
test = pd.read_csv("test_Neural.csv")

In [7]:
test['predicted_rating'] = model.predict([test.userIdCat, test.movieIdCat])
test.head()

,Unnamed: 0,userId,movieId,rating,timestamp,movieIdCat,userIdCat,predicted_rating
0,0,1,307,3.5,1256677221,304,0,4.469604
1,1,1,1590,2.5,1256677236,1538,0,3.297668
2,2,1,3424,4.5,1256677444,3336,0,4.222915
3,3,2,170,3.5,1192913581,168,1,2.990368
4,4,2,1296,4.5,1192913608,1267,1,3.957823


In [8]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(test['rating'], test['predicted_rating']))

0.6692100521015317


In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(test['predicted_rating'], test['rating']))
rmse

0.9065490452470936

In [0]:
test.head()
test.to_csv('test_Neural.csv')

In [10]:
test[['userId', 'movieId', 'rating', 'predicted_rating']]

,userId,movieId,rating,predicted_rating
0,1,307,3.5,4.469604
1,1,1590,2.5,3.297668
2,1,3424,4.5,4.222915
3,2,170,3.5,2.990368
4,2,1296,4.5,3.957823
...,...,...,...,...
5545780,283228,6874,4.0,4.675764
5545781,283228,7072,5.0,4.781937
5545782,283228,7121,4.0,4.724939
5545783,283228,7132,4.5,4.927802


In [0]:
test[['userId', 'movieId', 'rating', 'predicted_rating']].to_csv('NeuralNet_Prediction.csv') 

In [0]:
c = orig_df.userId.astype('category')

d = dict(enumerate(c.cat.categories))
print(d[0])

1


In [0]:
c = orig_df.movieId.astype('category')

d = dict(enumerate(c.cat.categories))
print(d[10959])

44761


In [0]:
orig_df['movieIdCat'] = orig_df.movieId.astype('category').cat.codes.values
orig_df['userIdCat'] = orig_df.userId.astype('category').cat.codes.values

In [0]:
orig_df[orig_df['movieId']==307]

In [0]:
orig_df

,userId,movieId,rating,timestamp,movieIdCat,userIdCat
0,1,307,3.5,1256677221,304,0
1,1,481,3.5,1256677456,477,0
2,1,1091,1.5,1256677471,1069,0
3,1,1257,4.5,1256677460,1229,0
4,1,1449,4.5,1256677264,1414,0
...,...,...,...,...,...,...
27753439,283228,8542,4.5,1379882795,7885,283227
27753440,283228,8712,4.5,1379882751,8029,283227
27753441,283228,34405,4.5,1379882889,10299,283227
27753442,283228,44761,4.5,1354159524,10959,283227


In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(orig_df, test_size=0.2)

In [0]:
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
n_users, n_movies = len(orig_df.userIdCat.unique()), len(orig_df.movieIdCat.unique())
n_latent_factors = 3

Using TensorFlow backend.


In [0]:
n_latent_factors_user = 5
n_latent_factors_movie = 8

movie_input = keras.layers.Input(shape=[1],name='Movie')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([movie_vec, user_vec], axis=-1)
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 8)         431120      Movie[0][0]                      
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         1416145     User[0][0]                       
____________________________________________________________________________________________

In [0]:
history = model.fit([train.userIdCat, train.movieIdCat], train.rating, epochs=9, verbose=1)

Epoch 1/9
22202755/22202755 [==============================] - 2980s 134us/step - loss: 0.6616
Epoch 2/9
22202755/22202755 [==============================] - 2975s 134us/step - loss: 0.6608
Epoch 3/9
22202755/22202755 [==============================] - 2985s 134us/step - loss: 0.6602
Epoch 4/9
22202755/22202755 [==============================] - 2984s 134us/step - loss: 0.6599
Epoch 5/9
22202755/22202755 [==============================] - 2959s 133us/step - loss: 0.6595
Epoch 6/9
22202755/22202755 [==============================] - 2963s 133us/step - loss: 0.6593
Epoch 7/9
22202755/22202755 [==============================] - 2949s 133us/step - loss: 0.6590
Epoch 8/9
22202755/22202755 [==============================] - 3073s 138us/step - loss: 0.6588
Epoch 9/9
22202755/22202755 [==============================] - 2954s 133us/step - loss: 0.6587


In [0]:
model.save('reco_model1.h5')

In [0]:
from keras.models import load_model
model = load_model('reco_model.h5')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(test['rating'], model.predict([test.userIdCat, test.movieIdCat])))

0.6664797103083782


In [0]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(test['rating'], model.predict([test.userIdCat, test.movieIdCat])))

0.6430068997195159


In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(test['rating'], model.predict([test.userIdCat, test.movieIdCat])))
rmse

0.8870687625367878

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(test['rating'], model.predict([test.userIdCat, test.movieIdCat])))
rmse

0.8857591336200893

In [0]:
orig_df['predicted_val'] = model.predict([orig_df.userIdCat, orig_df.movieIdCat])

In [0]:
predicted_val

array([[4.0789895],
       [3.5941312],
       [2.9966621],
       ...,
       [4.600849 ],
       [4.601058 ],
       [4.4601736]], dtype=float32)

In [0]:
orig_df.tail()

,userId,movieId,rating,timestamp,movieIdCat,userIdCat,predicted_val
27753439,283228,8542,4.5,1379882795,7885,283227,4.790021
27753440,283228,8712,4.5,1379882751,8029,283227,4.444968
27753441,283228,34405,4.5,1379882889,10299,283227,4.600849
27753442,283228,44761,4.5,1354159524,10959,283227,4.601058
27753443,283228,54286,4.5,1354159718,12034,283227,4.460174


In [0]:
import numpy as np
model.predict([np.array([283227]), np.array([7885])])
 	

array([[4.7900214]], dtype=float32)

In [0]:
import numpy as np
model.predict([np.array([0]), np.array([477])])

array([[3.5941312]], dtype=float32)

In [0]:
def PredictedRating(orig_df):
  return model.predict([orig_df.userIdCat, orig_df.movieIdCat])

orig_df.apply(PredictedRating, axis=1)

NameError: ignored

In [0]:
[test.userId, test.movieId]

[26612138    271754
 7482153      77053
 8342738      85884
 27059756    276197
 24283796    248326
              ...  
 27457060    280172
 10970865    112697
 14987213    153194
 24506388    250535
 9969474     102681
 Name: userId, Length: 5550689, dtype: int32, 26612138     1189
 7482153     15406
 8342738        40
 27059756     6205
 24283796     1041
             ...  
 27457060    15580
 10970865    10255
 14987213     6754
 24506388     2110
 9969474      2436
 Name: movieId, Length: 5550689, dtype: int32]

In [0]:
test.userId

26612138    271754
7482153      77053
8342738      85884
27059756    276197
24283796    248326
             ...  
27457060    280172
10970865    112697
14987213    153194
24506388    250535
9969474     102681
Name: userId, Length: 5550689, dtype: int32

In [0]:
test.movieId

26612138     1189
7482153     15406
8342738        40
27059756     6205
24283796     1041
            ...  
27457060    15580
10970865    10255
14987213     6754
24506388     2110
9969474      2436
Name: movieId, Length: 5550689, dtype: int32

In [0]:
model.predict([test.userId, test.movieId])

NameError: ignored

In [0]:
users_vs_rated_movies_dict = orig_df.groupby('userId')['movieId'].apply(list).to_dict()

In [0]:
movies = pd.read_csv('movies.csv')
all_movies_dict = pd.Series(movies['title'].values,index=movies['movieId']).to_dict()
users_vs_unrated_movies_dict = \
{k: list(set(all_movies_dict.keys()) - set(v)) for k,v in users_vs_rated_movies_dict.items()}
users_vs_unrated_movies_dict[1]

In [0]:
users_vs_unrated_movies_dict = \
{k: list(set(all_movies_dict.keys()) - set(v)) for k,v in users_vs_rated_movies_dict.items()}

NameError: ignored

In [0]:
users_vs_rated_movies_dict[1]

In [0]:
orig_df[['userId','movieId','rating','predicted_val']]

,userId,movieId,rating,predicted_val
0,1,307,3.5,4.078990
1,1,481,3.5,3.594131
2,1,1091,1.5,2.996662
3,1,1257,4.5,4.021384
4,1,1449,4.5,4.025838
...,...,...,...,...
27753439,283228,8542,4.5,4.790021
27753440,283228,8712,4.5,4.444968
27753441,283228,34405,4.5,4.600849
27753442,283228,44761,4.5,4.601058


In [0]:
unseen_df = pd.read_csv("users_vs_unrated_movies_long_format.csv")
unseen_df.head()

,users,unseen_movies
0,1,4.0
1,1,5.0
2,1,7.0
3,1,8.0
4,1,9.0


In [0]:
unseen_df.astype({'unseen_movies': 'int32'})
unseen_df.head()

,users,unseen_movies
0,1,4.0
1,1,5.0
2,1,7.0
3,1,8.0
4,1,9.0


In [0]:
unseen_df['movieIdCat1'] = unseen_df['unseen_movies'].apply(lambda x:orig_df['movieIdCat'][orig_df['movieId']==x].iloc[0])
unseen_df['userIdCat1'] = unseen_df['users'].apply(lambda x:orig_df['userIdCat'][orig_df['userId']==x].iloc[0])

KeyboardInterrupt: ignored

In [0]:
orig_df.tail()

,userId,movieId,rating,timestamp,movieIdCat,userIdCat,predicted_val
27753439,283228,8542,4.5,1379882795,7885,283227,4.790021
27753440,283228,8712,4.5,1379882751,8029,283227,4.444968
27753441,283228,34405,4.5,1379882889,10299,283227,4.600849
27753442,283228,44761,4.5,1354159524,10959,283227,4.601058
27753443,283228,54286,4.5,1354159718,12034,283227,4.460174


In [0]:
orig_df['movieIdCat'][orig_df['movieId']==54286].iloc[0]

12034

In [0]:
unseen_df.tail()

,users,unseen_movies,movieIdCat,userIdCat
5841169,610,32728.0,5860,609
5841170,610,163809.0,9386,609
5841171,610,32743.0,5861,609
5841172,610,98279.0,8039,609
5841173,610,98296.0,8040,609


In [0]:
c = orig_df.movieId.astype('category')

d = dict(enumerate(c.cat.categories))
print(d[12034])

54286
